In [1]:
!pip install pfrl@git+https://github.com/voidful/pfrl.git
!pip install textrl==0.2.15

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/voidful/pfrl.git to /tmp/pip-install-cwx7ldq_/pfrl_1c304f8bf4ea4b17b3dbc66e8850f7a2
  Running command git clone --filter=blob:none --quiet https://github.com/voidful/pfrl.git /tmp/pip-install-cwx7ldq_/pfrl_1c304f8bf4ea4b17b3dbc66e8850f7a2
  Resolved https://github.com/voidful/pfrl.git to commit bd679022d91902a3c88066d2646ad69b30d5684a
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/voidful/pfrl.git
!{sys.executable} -m pip install textrl==0.2.15

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/voidful/pfrl.git to /tmp/pip-req-build-z3hwhgkx
  Running command git clone --filter=blob:none -q https://github.com/voidful/pfrl.git /tmp/pip-req-build-z3hwhgkx
  Resolved https://github.com/voidful/pfrl.git to commit bd679022d91902a3c88066d2646ad69b30d5684a
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 721 kB 1.1 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pfrl: filename=pfrl-0.4.0-py3-none-any.whl size=155504 sha256=e2e425d15f324ee0e658bd425faf071cd578b72e0a90a8386a9db740a8d34541
  Stored in directory: /tmp/pip-ephem-wheel-cache-wr8k154h/wheels/4e/7a/0d/6d90ea0eecefee2bcf373177932fd54e84b41ef5190472e2db
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827638 sha256=7a521ac544af9615b51a5accc12fd4bd76f7b4a2

In [2]:
import sys
print(sys.executable)

/home/u9655801/miniconda3/envs/textrl/bin/python


In [3]:
!which pip

/home/u9655801/miniconda3/bin/pip


In [6]:
import pfrl
from textrl import TextRLEnv,TextRLActor
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelWithLMHead
import logging
import sys
import torch
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')
from NISQA.nisqa.NISQA_model import nisqaModel

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("lca0503/speech-chatgpt-base-nar-v2-epoch4-wotrans")
model = AutoModelForSeq2SeqLM.from_pretrained("lca0503/speech-chatgpt-base-nar-v2-epoch4-wotrans")
model.eval()
model.cuda()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(59481, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(59481, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [ ]:
sentiment = pipeline('sentiment-analysis',model="cardiffnlp/twitter-roberta-base-sentiment",tokenizer="cardiffnlp/twitter-roberta-base-sentiment",device=0,return_all_scores=True)

In [ ]:
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.CRITICAL)

In [ ]:
sentiment("dogecoin is bad")

In [ ]:
sentiment("dogecoin is bad")[0][0]['score']

In [ ]:
class MyRLEnv(TextRLEnv):
    def __init__(self, model, tokenizer, device="cuda", observation_input=[], max_length=100, compare_sample=2):
        super().__init__(model, tokenizer, observation_input, max_length, compare_sample)
        self.device = torch.device(device)
        self.tokenizer = tokenizer
        self.model = model
        self.observation_input = observation_input
        
        
    def get_reward(self, input_item, predicted_list, finish): # predicted will be the list of predicted token
        args = {
            'mode': 'predict_file',  # For example, 'predict_file', 'predict_dir', or 'predict_csv'
            'pretrained_model': 'weights/nisqa.tar',  # Name of the pretrained model file
            'deg': '/home/u9655801/hw/TextRL/NISQA/test.wav',  # Path to the speech file if mode is predict_file
            'data_dir': None,  # Folder with speech files if mode is predict_dir
            'output_dir': '/home/u9655801/hw/TextRL/NISQA/result',  # Folder to output results.csv
            'csv_file': None,  # Name of the csv file if mode is predict_csv
            'csv_deg': None,  # Column name in csv with file names/paths if mode is predict_csv
            'num_workers': 0,  # Number of workers for PyTorch's DataLoader
            'bs': 1,  # Batch size for predicting
            'ms_channel': None,  # Audio channel in case of a stereo file, if needed
        }

        if args['mode'] == 'predict_file':
            if args['deg'] is None:
                raise ValueError('--deg argument with path to input file needed')
        elif args['mode'] == 'predict_dir':
            if args['data_dir'] is None:
                raise ValueError('--data_dir argument with folder with input files needed')
        elif args['mode'] == 'predict_csv':
            if args['csv_file'] is None:
                raise ValueError('--csv_file argument with csv file name needed')
            if args['csv_deg'] is None:
                raise ValueError('--csv_deg argument with csv column name of the filenames needed')
            if args['data_dir'] is None:
                args['data_dir'] = ''
        else:
                raise NotImplementedError('--mode given not available')

        args['tr_bs_val'] = args['bs']
        args['tr_num_workers'] = args['num_workers']
        
        
        nisqa = nisqaModel(args)
        prediction = nisqa.predict()
        reward = int(prediction['mos_pred'].iloc[0])
        return reward

    def step(self, action):
        predicted, finish, predicted_str = self.predict(vocab_id=action)
        reward = self.get_reward(self.input_item, predicted, finish)
        # Assuming `action` is an index representing a choice of instruction or modification

        # Prepare the model input based on the chosen action
        # This involves selecting the instruction and possibly modifying voice features
        instruction, transcription, src_encodec_ids = self.prepare_input(action)

        # Generate voice output using the model
        # For simplicity, we're directly using a method that might encapsulate the model interaction
        layer_list = self.generate_voice(instruction, transcription, src_encodec_ids)

        # Convert model output to audio and evaluate it to calculate the reward
        encodec_code = convert_to_encode_code(self.tokenizer, layer_list)
        audio = synthesize_audio(encodec_code, self.device)
        reward = self.evaluate_audio(audio)  # This would be an external or predefined method

        # Check if the episode is done based on your criteria
        done = self.check_done()

        # Optionally update the environment state here

        return self.get_observation(), reward, done, {}
    
    def predict(self, vocab_id):
        predicted_list = {}
        predicted_list_end = {}
        with torch.inference_mode():
            for i, (v_id, predicted, predicted_end) in enumerate(zip(vocab_id, self.predicted, self.predicted_end)):
                predicted_list_end[i] = False
                if not predicted_end:
                    pred_word = self.actions[v_id]
                    if pred_word in self.gen_stop_toks \
                            or len(pred_word) < 1 \
                            or len(predicted) > self.env_max_length:
                        predicted_list_end[i] = True
                        predicted_list[i] = [pred_word]
                    else:
                        predicted_list[i] = [pred_word]
                else:
                    predicted_list_end[i] = True
                    predicted_list[i] = ['']

            for i, (l, e) in enumerate(zip(predicted_list.values(), predicted_list_end.values())):
                self.predicted[i] = self.predicted[i] + l
                self.predicted_end[i] = e

            return self.predicted, all(self.predicted_end), [self.tokenizer.convert_tokens_to_string(i) for i in
                                                             self.predicted]
    def reset(self):
        # Reset or initialize the environment state
        # This could involve loading or selecting a new dataset entry for instructions and voice features

        self.current_instruction, self.current_transcription, self.current_src_encodec_ids = self.select_new_instruction()

        # Prepare the initial observation based on the new instruction
        # This step depends on how you define an observation in your environment
        initial_observation = self.prepare_observation(self.current_instruction, self.current_transcription)

        return initial_observation


In [ ]:
input_text = [{}]
input_audio = ""

observaton_list = [{'input':'i think dogecoin is'}]

In [ ]:
env = MyRLEnv(model, tokenizer, observation_input=observaton_list, compare_sample=1)
actor = TextRLActor(env,model,tokenizer,optimizer='adamw',
                    temperature=0.8,
                    top_k=100,
                    top_p=0.85,)
agent = actor.agent_ppo(update_interval=50, minibatch_size=3, epochs=10,lr=3e-4)
print(actor.predict(observaton_list[0]))

pfrl.experiments.train_agent_with_evaluation(
    agent,
    env,
    steps=3000,
    eval_n_steps=None,
    eval_n_episodes=1,       
    train_max_episode_len=100,  
    eval_interval=10,
    outdir='checkpoint', 
)
agent.load("./checkpoint/best")
print(actor.predict(observaton_list[0]))